# Explore and cluster the neighborhoods in Toronto

## Bring datafrme from previous question

In [1]:
%store -r df5
df5

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


import essential library

In [2]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('done')

done


## Explore the Neighbourhood

### Focusing the explore in Borough thats contain the word 'Toronto'

Create the dataframe that Borough contain the word 'Toronto'

In [3]:
Toronto_data = df5[df5['Borough'].str.find('Toronto') != -1].reset_index(drop=True) #.str.find('Text') will eqau; -1 if string not fond 
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Create the Map of Toronto, Canada

import library to create Map

In [4]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [5]:
import folium
print ('Done')

Done


From wikipidia The geograpical coordinate of Toronto, Canada are N 43.6532, W 79.3832 .

In [6]:
#define coridinate of Toronto
latitude = 43.6532
longitude = -79.3832 

#create map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
map_Toronto

Add marker for each postal code to the map

In [7]:
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

map_Toronto

#### Using the Foursquare API to explore the neighborhoods around

Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'BISODONBIY3BLADK12TAYRC2V3RT3FPLXDKCC0ZTLCXEMCJN' # My Foursquare ID
CLIENT_SECRET = 'LINC0NSMHE5CAK10DMXU1Q2KOHAIOZ1Z2VYN2ZWWWW34JIDO' # My Foursquare Secret
VERSION = '20190625' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BISODONBIY3BLADK12TAYRC2V3RT3FPLXDKCC0ZTLCXEMCJN
CLIENT_SECRET:LINC0NSMHE5CAK10DMXU1Q2KOHAIOZ1Z2VYN2ZWWWW34JIDO


#### Explorer top 20 venues that are in each postal code within a radius of 500 meters.

##### create a function to repeat the same process to all postcode in Toronto and create Dataframe

In [9]:
LIMIT = 20 # limit of number of venues from Foursquare

radius = 500 # radius around cordinate unit 'meter' 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL to Foursquare
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called Toronto_venues by running the above function on each postal code.

In [10]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Postcode'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

print(Toronto_data.shape)


M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y
(38, 5)


In [11]:
Toronto_venues.head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Count the unique catagory

In [12]:
len(Toronto_venues['Venue Category'].unique()) #Count uniuqe Category 

164

## Analyze Each Postcode

create dataframe that show the existance of each catagory in each row

In [13]:
# one hot encoding to create dummy value. If the row contain category, value equl 1
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column to dataframe
Toronto_onehot['Postcode'] = Toronto_venues['Postcode'] 

# move postcode column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Postcode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by Postcode and taking the mean of the frequency of occurrence of each category

In [14]:
Toronto_grouped = Toronto_onehot.groupby('Postcode').mean().reset_index()
Toronto_grouped

,Postcode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.20,0.00,0.000000,0.00,0.000000
1,M4K,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.05,0.00,0.000000,0.00,0.050000
2,M4L,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
3,M4M,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.050000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
4,M4N,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
5,M4P,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
6,M4R,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.062500
7,M4S,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.0,0.05,0.00,0.00,0.05,0.00,0.00,0.000000,0.00,0.000000
8,M4T,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.00,0.00,...,0.5,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
9,M4V,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.071429,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.071429,0.00,0.000000


Create a function to sort the venues in descending order.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
print('done')

done


Create the new dataframe and display the top 5 venues for each neighborhood.

In [16]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = Toronto_grouped['Postcode']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Trail,Health Food Store,Pub,Other Great Outdoors,Neighborhood
1,M4K,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore
2,M4L,Park,Gym,Steakhouse,Ice Cream Shop,Fish & Chips Shop
3,M4M,Coffee Shop,Bakery,Café,Italian Restaurant,Ice Cream Shop
4,M4N,Bus Line,Park,Swim School,Creperie,Dog Run


## Cluster the postcode

Using K-mean cluster to clustering

In [17]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 4, 1])

create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each Postcode
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Other Great Outdoors,Neighborhood
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Park,Gym,Steakhouse,Ice Cream Shop,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bakery,Café,Italian Restaurant,Ice Cream Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Creperie,Dog Run


visualize each lable on folium map

In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Postcode'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True) #create text in pop up lable
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine  Each Clusters

### Cluster 1

In [20]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Central Toronto,0,Bus Line,Park,Swim School,Creperie,Dog Run


### Cluster 2

In [21]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,1,Trail,Health Food Store,Pub,Other Great Outdoors,Neighborhood
1,East Toronto,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore
2,East Toronto,1,Park,Gym,Steakhouse,Ice Cream Shop,Fish & Chips Shop
3,East Toronto,1,Coffee Shop,Bakery,Café,Italian Restaurant,Ice Cream Shop
5,Central Toronto,1,Gym,Breakfast Spot,Hotel,Food & Drink Shop,Park
6,Central Toronto,1,Coffee Shop,Yoga Studio,Bagel Shop,Park,Chinese Restaurant
7,Central Toronto,1,Dessert Shop,Sushi Restaurant,Italian Restaurant,Gym,Indian Restaurant
9,Central Toronto,1,Coffee Shop,Pub,Pizza Place,Supermarket,Vietnamese Restaurant
11,Downtown Toronto,1,Café,Restaurant,Gift Shop,Indian Restaurant,Pet Store
12,Downtown Toronto,1,Gastropub,Japanese Restaurant,Park,Pizza Place,Creperie


### Cluster 3

In [22]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,2,Garden,Ice Cream Shop,Yoga Studio,Creperie,Dog Run


### Cluster 4

In [23]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Downtown Toronto,3,Park,Playground,Trail,Cosmetics Shop,Discount Store
23,Central Toronto,3,Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio


### Cluster 5

In [24]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,4,Playground,Tennis Court,Cosmetics Shop,Dog Run,Discount Store
